# AI Blog Generator

Justin Cripps, Ethan Hodge, Philip Lou, Tina Tissington

Make sure to replace `'YOUR_API_KEY'` with your Gemini API Key

In [8]:
import dspy

# Setting up Gemini as our Google LLM

gemini = dspy.Google("models/gemini-1.0-pro",
                        api_key='YOUR_API_KEY',
                        temperature=0)
                        
dspy.settings.configure(lm=gemini, max_tokens=1024)

This program implements the dspy library to program the LLM rather than prompt it.

In [2]:
from dspy import Signature, InputField, OutputField
from pydantic import BaseModel, Field
from typing import List

To interact with the LLM, we can create classes that represent aspects of our website we would like to create. The class below creates the skeleton of producing an image. 

In [3]:
class AIImage(BaseModel):
    """A single generated image."""

    prompt: str = Field(desc="The prompt used to generate the image.")
    url: str = Field(
        desc="The URL of the generated image.", default="./img/placeholder.webp"
    )

Now we can declare classes for the Webpage, Website, and Style (CSS):

In [13]:
class Webpage(BaseModel):
    """A single page in a website."""

    title: str = Field(desc="The page's title.")
    id: str = Field(desc=f"Lowercase of {title}")
    blurb: str = Field(desc="Catchhy blurb or quote about the page.")
    image: AIImage = Field(desc="A nice AI generated image to accompany the page.")
    content: str = Field(desc="Paragraph about the site")
    bullets: List[str] = Field(
        desc="Up to 5 bullet points of concise, relevant content."
    )

    def to_html(self):
        html_output = f'<h2 id="{self.id}">{self.title}</h2><p>{self.blurb}</p><img src="{self.image.url}" width="400" alt="{self.image.prompt}"><p>{self.content}</p><ul>'
        for bullet in self.bullets:
            html_output += f"<li>{bullet}</li>"
        html_output += f"</ul>"
        return html_output

class Website(BaseModel):
    """A complete website with a title, description, and content."""

    title: str = Field(desc="The blog webpage's title in HTML.")
    webpages: List[Webpage] = Field(desc="The pages that make up the website.")
    navbar: str = Field(desc="HTML and CSS to create a dynamic nav bar for each section of the site using their ids.")
    footer: str = Field(desc="Footer for the website.")

    def to_html(self):
        html_output = f"<head><link rel='stylesheet' href='main.css'></head><h1>{self.title}</h1><div>{self.navbar}</div>"
        for webpages in self.webpages:
            html_output += webpages.to_html()
        html_output += f"<footer>{self.footer}</footer>"
        return html_output
    
class Style(BaseModel):
    """A complete website style."""

    css: str = Field(desc="Create custom css for the website to make it feel like a professional blog.")

    def to_html(self):
        html_output = f'{self.css}'
        return html_output


The last class we create is a website creator class which we can call to create our blog:

In [6]:
class WebsiteCreator(Signature):
    """Create content for a great blog website."""
    
    website_subject: str = InputField(desc="The subject of the website.")
    website_content: Website = OutputField(desc="The complete website content.")
    website_styling: Style = OutputField(desc="The complete website styling")

Call the website_creator to create a website about "Engineering at the University of Guelph". If you get an error, refer to the readme and try changing the subect and running again.

In [15]:
from dspy.functional import TypedPredictor

website_creator = TypedPredictor(WebsiteCreator)
blog_website = website_creator(website_subject="Blog About Engineering at the University of Guelph.")

Finally save the html and css generated and display it in the noteboook:

In [16]:
from pprint import pprint

pprint(blog_website.website_content.to_html())

with open("home.html", "w") as file:
    file.write(blog_website.website_content.to_html())

with open("css/main.css", "w") as file:
    file.write(blog_website.website_styling.to_html())

from IPython.display import display, HTML
display(HTML(blog_website.website_content.to_html()))

("<head><link rel='stylesheet' href='main.css'></head><h1>Engineering at the "
 'University of Guelph</h1><div><nav>\n'
 '  <ul>\n'
 '    <li><a href="#home">Home</a></li>\n'
 '    <li><a href="#about">About</a></li>\n'
 '    <li><a href="#contact">Contact</a></li>\n'
 '  </ul>\n'
 '</nav></div><h2 id="home">Home</h2><p>Welcome to the Engineering at the '
 'University of Guelph blog!</p><img src="./img/home.webp" width="400" alt="A '
 'group of students working on a project in a lab."><p>This blog is a space '
 'for students, faculty, and staff to share their experiences, insights, and '
 'research in the field of engineering at the University of '
 'Guelph.</p><ul><li>We will be posting about a variety of topics, '
 'including:</li></ul><h2 id="about">About</h2><p>A little bit about '
 'us.</p><img src="./img/about.webp" width="400" alt="A photo of the '
 'University of Guelph campus."><p>The Faculty of Engineering at the '
 'University of Guelph is one of the leading engineering scho